In [50]:
# !pip install prophet

In [49]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go

# Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

# Split data into train and test
train_df = df[:-12]
test_df = df[-12:]

# Prophet Forecast
prophet_model = Prophet()
prophet_model.fit(train_df)
future = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future)
prophet_forecast.set_index('ds', inplace=True)

# Rolling Moving Average Forecast
window = 12
rolling_forecast_values = list(train_df['y'].iloc[-window:].values)  # start with last 5 real values
forecast_dates = test_df['ds'].values
rolling_preds = []

for i in range(len(forecast_dates)):
    avg = sum(rolling_forecast_values[-window:]) / window
    rolling_preds.append(avg)
    rolling_forecast_values.append(avg)  # append prediction for use in next rolling window

# Create DataFrame
rolling_forecast = pd.DataFrame({
    'ds': forecast_dates,
    'yhat': rolling_preds
}).set_index('ds')

# ------------------ Plotting ------------------ #
fig = go.Figure()

# Plot Actuals
actuals_end = test_df['ds'].max()
actuals_df = df[df['ds'] <= actuals_end]
actuals_df = actuals_df[actuals_df['ds'] < '1960-01-01']

fig.add_trace(go.Scatter(
    x=actuals_df['ds'], y=actuals_df['y'],
    mode='lines+markers',
    name='Actuals',
    line=dict(color='black', width=2),
    marker=dict(size=4)
))

# Prophet Forecast
fig.add_trace(go.Scatter(
    x=prophet_forecast.index, y=prophet_forecast['yhat'],
    mode='lines',
    name='Prophet Forecast',
    line=dict(color='blue', width=2)
))

# Prophet Confidence Interval
fig.add_trace(go.Scatter(
    x=pd.concat([
        pd.Series(prophet_forecast.index),
        pd.Series(prophet_forecast.index[::-1])
    ]),
    y=pd.concat([
        prophet_forecast['yhat_upper'],
        prophet_forecast['yhat_lower'][::-1]
    ]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='Prophet Confidence Interval'
))

# Rolling Moving Average Forecast
fig.add_trace(go.Scatter(
    x=rolling_forecast.index, y=rolling_forecast['yhat'],
    mode='lines',
    name='Rolling Moving Average Forecast',
    line=dict(color='green', width=2, dash='dot')
))

# Forecast start vertical line
forecast_start = test_df['ds'].iloc[0].timestamp() * 1000
fig.add_vline(
    x=forecast_start,
    line_width=2, line_dash="dot", line_color="gray",
    annotation_text="Forecast Start", annotation_position="top right"
)

# Layout settings
fig.update_layout(
    title='📈 Forecast Comparison: Prophet vs Rolling Moving Average',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.5)', bordercolor='black'),
    template='plotly_white',
    height=500
)

fig.show()


23:08:32 - cmdstanpy - INFO - Chain [1] start processing
23:08:33 - cmdstanpy - INFO - Chain [1] done processing
